In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset-mindcase/blade_runner.pdf


In [2]:
import torch
torch.cuda.empty_cache()

### NOTE:
#### I'm not using the API for LLAMA because the model exceeds the Memory limit of kaggle GPU

In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()

In [4]:
EMB_INSTRUCTOR_XL = "hkunlp/instructor-xl"
EMB_SBERT_MPNET_BASE = "sentence-transformers/all-mpnet-base-v2"

In [5]:
LLM_LLAMA = "meta-llama/Llama-2-7b-chat-hf"
LLM_FLAN_T5_BASE = "google/flan-t5-base"
LLM_FLAN_T5_XXL = "google/flan-t5-xxl"
LLM_FLAN_T5_XL = "google/flan-t5-xl"
LLM_FASTCHAT_T5_XL = "lmsys/fastchat-t5-3b-v1.0"
LLM_FLAN_T5_SMALL = "google/flan-t5-small"
LLM_FLAN_T5_LARGE = "google/flan-t5-large"
LLM_FALCON_SMALL = "tiiuae/falcon-7b-instruct"

In [6]:
config = {"persist_directory":None,
          "load_in_8bit":False,
          "embedding" : EMB_SBERT_MPNET_BASE,
          "llm":LLM_FLAN_T5_LARGE,
          }

In [7]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 29.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
      Successfully uninstalled orjson-3.9.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cub

In [8]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.5 MB/s eta 0:00:00a 0:00:01


In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from transformers import pipeline
import torch
from transformers import AutoTokenizer

def create_sbert_mpnet():
        device = "cuda" if torch.cuda.is_available() else "cpu"
        return HuggingFaceEmbeddings(model_name=EMB_SBERT_MPNET_BASE, model_kwargs={"device": device})


def create_model(load_in_8bit=False):
        # Wrap it in HF pipeline for use with LangChain
        model=LLM_FLAN_T5_LARGE
        tokenizer = AutoTokenizer.from_pretrained(model)
        return pipeline(
            task="text2text-generation",
            model=model,
            tokenizer = tokenizer,
            max_new_tokens=200,
#             model_kwargs={"device_map": "auto", "load_in_8bit": load_in_8bit, "max_length": 512, "temperature": 0.}
        )


if config["embedding"] == EMB_SBERT_MPNET_BASE:
    embedding = create_sbert_mpnet()

load_in_8bit = config["load_in_8bit"]
if config["llm"] == LLM_FLAN_T5_LARGE:
    llm = create_model(load_in_8bit=load_in_8bit)

2024-04-19 13:54:48.090532: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-19 13:54:48.090654: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-19 13:54:48.225320: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [10]:
!pip install langchain langchain-text-splitters pdfplumber tiktoken chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 16.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 71.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 47.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/69

In [11]:
# Load the pdf
from langchain.vectorstores import Chroma
from langchain_community.document_loaders.pdf import PDFPlumberLoader
from langchain_text_splitters import CharacterTextSplitter, TokenTextSplitter
pdf_path = "/kaggle/input/dataset-mindcase/blade_runner.pdf"
loader = PDFPlumberLoader(pdf_path)
documents = loader.load()

# Split documents and create text snippets
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=10, encoding_name="cl100k_base")  # This the encoding for text-embedding-ada-002
texts = text_splitter.split_documents(texts)

persist_directory = config["persist_directory"]
vectordb = Chroma.from_documents(documents=texts, embedding=embedding, persist_directory=persist_directory)

In [14]:
from langchain_community.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate
hf_llm = HuggingFacePipeline(pipeline=llm)
retriever = vectordb.as_retriever(search_kwargs={"k":4})
qa = RetrievalQA.from_chain_type(llm=hf_llm, chain_type="stuff",retriever=retriever)

# Defining a default prompt for flan models
if config["llm"] == LLM_FLAN_T5_SMALL or config["llm"] == LLM_FLAN_T5_BASE or config["llm"] == LLM_FLAN_T5_LARGE:
    question_t5_template = """
    context: {context}
    question: {question}
    answer: 
    """
    QUESTION_T5_PROMPT = PromptTemplate(
        template=question_t5_template, input_variables=["context", "question"]
    )
    qa.combine_documents_chain.llm_chain.prompt = QUESTION_T5_PROMPT

In [15]:
question = "Explain the theme of the movie?"
qa.combine_documents_chain.verbose = True
qa.return_source_documents = True
qa({"query":question,})

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Token indices sequence length is longer than the specified maximum sequence length for this model (1348 > 512). Running this sequence through the model will result in indexing errors




> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': 'Explain the theme of the movie?',
 'result': 'Replicants',
 'source_documents': [Document(page_content='B L A D E R U N N E R 2 0 4 9\nstory by\nHampton Fancher\nscreenplay by\nHampton Fancher\nand\nMichael Green\nFINAL SHOOTING SCRIPT', metadata={'CreationDate': 'D:20240418012249', 'Creator': 'PDFium', 'Producer': 'PDFium', 'file_path': '/kaggle/input/dataset-mindcase/blade_runner.pdf', 'page': 0, 'source': '/kaggle/input/dataset-mindcase/blade_runner.pdf', 'total_pages': 109}),
  Document(page_content='13.\nWalking up 80 stories. Stiff from the fight. An OLD WOMAN\nbumps into him, expecting him to get out of her way. No one\nhe passes the least glad to see a Replicant.\nINT. HALLWAY.\nK crosses the hall. ALL his NEIGHBORS’ DOORS are open like\nmarket stalls letting in air and letting out CHILDREN. Every\nhome thickly filled, like a hoarder’s garage. Some host\nSHOPS. FOOD STALLS. Every floor like a town square.\nK walks past and to his apartment. A BOY, brown-faced,\nmissi

In [16]:
question = "Who are the characters?"
qa.combine_documents_chain.verbose = True
qa.return_source_documents = True
qa({"query":question,})



> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': 'Who are the characters?',
 'result': 'K, Joi',
 'source_documents': [Document(page_content='99.\nHer eyes sparkle with the future she sees.\nBut K is focused only on one word. He feels it like a bullet.\nK\nShe?\nFreysa nods, confused by his confusion on the point.\nFREYSA\nCourse. Rachael have a daughter. A baby\ngirl.\nAn infinite beat. Time shatters. Gravity upends.\nK shakes his head. No.\nFREYSA (cont’d)\nWith my own eye. I see her come, I dress\nher in blue when time she go.\nK\nNo. A girl died. It was a boy you hid.\nFreysa shakes her head knowingly.\nFREYSA\nJust a piece a puzzle.\nA wave of nausea rocks K. The crushing realization slowly\ntaking root... eating away at his new soul...\nHe was wrong about who he is.\nON K. At the moment he sees it. The connection forming.\nINSERT CUT: ANA. STARING, CONFUSED, MOVED BY WHAT SHE SEES\nIN K’S MEMORY. A MOMENT OF RECOGNITION.\nANA\nNo one invented that. It was a real\nmoment. Someone lived this.\nINSERT CUT: THE DENABASE S

In [17]:
question = "How many male and female characters are in the movie?"
qa.combine_documents_chain.verbose = True
qa.return_source_documents = True
qa({"query":question,})



> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': 'How many male and female characters are in the movie?',
 'result': '10',
 'source_documents': [Document(page_content='95.\nNIANDER WALLACE (cont’d)\nDo not be afraid.\nA WOMAN steps across the path over the water. The woman\nsteps into the light. Deckard sees...\nRACHAEL\nPrecisely as when he met her. As she was in the holo. Down\nto the lashes. Flesh and blood. Young again. Remade.\nAuthentic. Inauthentic.\nON DECKARD. Breath gone. Wallace presents his offer.\nNIANDER WALLACE (cont’d)\nAn Angel. Made again. For you.\nDeckard fights the urge, can’t help but look. A perfect\nrecreation of all he ever wanted is right in front of him.\nIn the flesh. Instantly filled with longing. With disgust.\nRachael steps close to Deckard. Lifts his chin. So they are\nface to face. We are transported back decades in seconds.\nRACHAEL\nDid you miss me?\nDeckard says nothing.\nRACHAEL (cont’d)\nDon’t you love me?\nThe mirage utterly real and convincing. Deckard strains\nagainst pain of loss. S

In [18]:
question = "Does the script pass the Bechdel test?"
qa.combine_documents_chain.verbose = True
qa.return_source_documents = True
qa({"query":question,})



> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': 'Does the script pass the Bechdel test?',
 'result': 'yes',
 'source_documents': [Document(page_content='45.\nHe quiets. Just then -- K sees something in the dataflow.\nSits up suddenly.\nK (cont’d)\nThere. Hold.\nThe snowstorm of LETTERS FREEZES. K moves close. Sees\nsomething remarkable in the chaos.\nK (cont’d)\nPut up 4847 and 2181. Side by side.\nThe datasets comply. All other information falls away as two\nchains of DNA move side by side. Then OVERLAP so we can see:\nTHE LETTERS OF EACH OF THEM MATCH EXACTLY.\nK (cont’d)\nThey’re identical... Translate.\nIt READS and translates the THE DNA CHAINS. Sorting the\nLETTERS into codon groups of THREE, which string into LONG\nCHAINS (proteins), and then finally into WORDS.\nONE SET IS LABELED FEMALE 2181, DECEASED. THE OTHER: MALE\n4847, NO ADDITIONAL DATA.\nK (cont’d)\nA boy and a girl. It’s not possible.\nTwo people can’t have identical DNA. One\nof these isn’t real. It’s a copy...\nShow all telemetry.\nK manipulates the dat

In [19]:
question = "What is the role of Deckard in the movie?"
qa.combine_documents_chain.verbose = True
qa.return_source_documents = True
qa({"query":question,})



> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': 'What is the role of Deckard in the movie?',
 'result': 'Officer KD6',
 'source_documents': [Document(page_content='106.\nEXT. SEA WALL BASE.\nDeckard grabs hold of K before he is swept out into the sea.\nThey find their footing, their breath. Deckard nods to K.\nHe’s all right. They look out to see...\nTHE TRANSPORT VAN SINKS BENEATH THE SURFACE. GONE.\nK and Deckard together, further up the sea wall. Freezing.\nBut alive. Under the blood orange sky of a new day coming.\nThey sit. Find their breath.\nDECKARD\nYou should’ve let me die.\nK\nYou did. You drowned in the ocean.\nYou’re free. Free to meet your daughter.\nTough as he is -- Deckard is overcome with emotion for the\nfirst time in decades. Just hearing the word. Brings\nhimself to say it.\nDECKARD\nDaughter...?\nEverything he longed for. In a word. In time he breathes.\nCUT TO:\nEXT. SKIES ABOVE LOS ANGELES. DAWN.\nThe grand CITYSCAPE. Under a blanket of snow. Clean as a\nfresh start.\nEXT. SNOWY FIELD. DAWN.\nK and D